<a href="https://colab.research.google.com/github/amodkala/pmpp/blob/master/matmul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpc35z04m8".


In [6]:
%%cuda

/*
    This is the most naive implementation of a matmul kernel
*/

#include <stdio.h>

__global__ void matMulKernel(float *A, float *B, float *C, size_t m, size_t n, size_t k) {

	// calculate row and column indices for this thread
	int row = blockIdx.x * blockDim.x + threadIdx.x;
	int col = blockIdx.y * blockDim.y + threadIdx.y;

	if (row >= m || col >= k) {
		return;
	}

	float sum = 0.0f;
	for (int i = 0; i < n; ++i) {
		sum += A[row * n + i] * B[i * k + col];
	}

	C[row * k + col] = sum;
}

void matMul(float *A, float *B, float *C, size_t m, size_t n, size_t k) {

	// allocate device memory for input and output matrices
	float *A_d, *B_d, *C_d;

	size_t A_size = m * n * sizeof(float);
	size_t B_size = n * k * sizeof(float);
	size_t C_size = m * k * sizeof(float);

	cudaMalloc((void **) &A_d, A_size);
	cudaMalloc((void **) &B_d, B_size);
	cudaMalloc((void **) &C_d, C_size);

	// copy input matrices to device
	cudaMemcpy(A_d, A, A_size, cudaMemcpyHostToDevice);
	cudaMemcpy(B_d, B, B_size, cudaMemcpyHostToDevice);

	// declare grid/block dimensions
	int n_threads = 16;
	int grid_rows = (m + n_threads - 1) / n_threads;
    	int grid_cols = (k + n_threads - 1) / n_threads;

    	dim3 grid(grid_rows, grid_cols);
    	dim3 block(n_threads, n_threads);

	// call matMul kernel
    	matMulKernel<<<grid, block>>>(A_d, B_d, C_d, m, n, k);

	// copy output matrix to host
	cudaMemcpy(C, C_d, C_size, cudaMemcpyDeviceToHost);

	// free device memory
	cudaFree(A_d);
	cudaFree(B_d);
	cudaFree(C_d);
}

int main(int argc, char* argv[]) {

	size_t m = 3, n = 6, k = 4;

	// declare new matrices
	float *A = new float[m * n];
	float *B = new float[n * k];
	float *C = new float[m * k];

	int e;

	printf("Initializing matrix A: \n");
	for (int i = 0; i < m; ++i) {
		for (int j = 0; j < n; ++j) {
			e = (i*n) + j;
			A[e] = (float)e;
			printf("%f ", (float)e);
		}
		printf("\n");
	}

	printf("Initializing matrix B: \n");
	for (int i = 0; i < n; ++i) {
		for (int j = 0; j < k; ++j) {
			e = (i*k) + j;
			B[e] = (float)e;
			printf("%f ", (float)e);
		}
		printf("\n");
	}

	// call matMul function
	matMul(A, B, C, m, n, k);

	printf("Matrix C: \n");
	for (int i = 0; i < m; ++i) {
		for (int j = 0; j < k; ++j) {
			printf("%f ", C[(i*k) + j]);
		}
		printf("\n");
	}

	// free memory
	delete[] A;
	delete[] B;
	delete[] C;

	return 0;

}

Initializing matrix A: 
0.000000 1.000000 2.000000 3.000000 4.000000 5.000000 
6.000000 7.000000 8.000000 9.000000 10.000000 11.000000 
12.000000 13.000000 14.000000 15.000000 16.000000 17.000000 
Initializing matrix B: 
0.000000 1.000000 2.000000 3.000000 
4.000000 5.000000 6.000000 7.000000 
8.000000 9.000000 10.000000 11.000000 
12.000000 13.000000 14.000000 15.000000 
16.000000 17.000000 18.000000 19.000000 
20.000000 21.000000 22.000000 23.000000 
Matrix C: 
220.000000 235.000000 250.000000 265.000000 
580.000000 631.000000 682.000000 733.000000 
940.000000 1027.000000 1114.000000 1201.000000 

